#Read data from processed dataframe

In [1]:
import pandas as pd

from google.colab import drive

In [2]:
drive.mount('/content/drive')
home_dir = "/content/drive/My Drive/Colab Notebooks/hackaton/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
df = pd.read_csv(home_dir+'data_cleaned4_combined.csv')
df['combined'] = df['combined'].fillna('')

## User input text preparation

In [4]:
! pip install pymorphy2

In [5]:
import re

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

import pymorphy2

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stopwords_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [8]:
def clean_text(text: str) -> str:
    text = re.sub(r'((http|ftp)\S+)|[^a-zа-яё\s/-]|(-{2,})', '', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_stopwords(tokens: list, stopwords=None) -> list:
    if not stopwords:
        return tokens

    stopwords = set(stopwords)
    tokens = [tok for tok in tokens if tok not in stopwords and len(tok)>2]

    return tokens

def text_preparation(text: str, to_str=True):
    text = clean_text(text)
    tokens = word_tokenize(text, language='russian')
    tokens = remove_stopwords(tokens, stopwords_ru)
    tokens = [morph.parse(tok)[0].normal_form for tok in tokens]

    return ' '.join(tokens) if to_str else tokens

## Imitation user input

In [9]:
# @title

import ipywidgets as widgets
from IPython.display import display
import numpy as np

# DataFrame
df_copy = df
def change_df():
    global df_copy
    df_copy = df

    if city != '':
        df_copy = df_copy[df_copy['city'] == city]

    if education_form != '':
        df_copy = df_copy[df_copy['education_form'] == education_form]

    df_copy.reset_index(drop=True, inplace=True)

# City
city = ''
cities = [''] + df['city'].value_counts().index.values.tolist()
Dropdown_cities = widgets.Dropdown(
    options=cities,
    description='Город',
)
output = widgets.Output()

def onchange_cities(change):
    global city
    city = change['new']
    change_df()

Dropdown_cities.observe(onchange_cities, names='value')
display(Dropdown_cities)

about = str(input("Напишите о себе: "))
activities = str(input("Напишите о своей деятельности: "))
books = str(input("Укажите ваши любимые книги: "))
games = str(input("Укажите ваши любимые игры: "))
interests = str(input("Напишите о своих интересах: "))

# Education forms
education_form = ''
education_forms = [''] + df['education_form'].value_counts().index.values.tolist()
Dropdown_education_forms = widgets.Dropdown(
    options=education_forms,
    description='Форма обучения',
)
output = widgets.Output()

def onchange_education_forms(change):
    global education_form
    education_form = change['new']
    change_df()

Dropdown_education_forms.observe(onchange_education_forms, names='value')
display(Dropdown_education_forms)

# Education statuses
education_status = ''
education_statuses = [''] + df['education_status'].value_counts().index.values.tolist()
Dropdown_education_statuses = widgets.Dropdown(
    options=education_statuses,
    description='Статус образования',
)
output = widgets.Output()

def onchange_education_statuses(change):
    global education_status
    education_status = change['new']

Dropdown_education_statuses.observe(onchange_education_statuses, names='value')
display(Dropdown_education_statuses)

#Top of recomendations
top_n = 5
tops = [5, 10, 15, 20]
Dropdown_tops = widgets.Dropdown(
    options=tops,
    description='Кол-во рекомендаций',
)
output = widgets.Output()

def onchange_education_statuses(change):
    global top_n
    top_n = change['new']

Dropdown_tops.observe(onchange_education_statuses, names='value')
display(Dropdown_tops)

user_input = text_preparation(about + ' ' + activities + ' ' + books + ' ' + games + ' ' + interests)

Dropdown(description='Город', options=('', 'Москва', 'Санкт-Петербург', 'Екатеринбург', 'Казань', 'Новосибирск…

Напишите о себе: программист
Напишите о своей деятельности: программирование
Укажите ваши любимые книги: грокаем алгоритмы, чистый код
Укажите ваши любимые игры: counter-strike, fifa
Напишите о своих интересах: программирование, шахматы, кататься на сноуборде


Dropdown(description='Форма обучения', options=('', 'Очное отделение', 'Заочное отделение', 'Очно-заочное отде…

Dropdown(description='Статус образования', options=('', 'Выпускник', 'Студент', 'Выпускник (специалист)', 'Сту…

Dropdown(description='Кол-во рекомендаций', options=(5, 10, 15, 20), value=5)

In [10]:
user_input

'программист программирование грокать алгоритм чистый код counter-strike fifa программирование шахматы кататься сноуборд'

#TF-IDF recommendations

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [55]:
tfidf = TfidfVectorizer(stop_words=stopwords_ru, lowercase=False)

In [95]:
def get_tfidf_recommendations():
    recommendations = df_copy.loc[df_copy['combined']!='', ['faculty_name', 'actual_university_name', 'combined']].reset_index(drop=True)

    user_tfidf = tfidf.fit_transform([user_input]+recommendations['combined'].tolist())

    # вычисление косинусного сходства между введенными данными и всеми пользователями
    cosine_sim_user = linear_kernel(user_tfidf[0], user_tfidf).flatten()
    # получение индекса пользователя с наибольшим косинусным сходством
    similar_user_indices = cosine_sim_user[1:].argsort(axis=0)[-1:-top_n-1:-1]
    # получение рекомендаций университета и факультета на основе наиболее похожего пользователя
    recommendations = recommendations.loc[similar_user_indices, :]

    return recommendations

In [118]:
get_tfidf_recommendations()

,faculty_name,actual_university_name,combined
17003,Электротехнический,ЮУрГУ (НИУ),программирование
7292,Факультет автоматики и информационных технологий,СамГТУ,программирование дизайн
15268,Лечебный,ДВГМУ (бывш. ХГМИ),web-программирование
12779,Информационных технологий,ВлГУ им. А. Г. и Н. Г. Столетовых,counter-strike
18310,Экономический,Филиал ВЭПИ,counter strike
17447,Электротехнический,СамГТУ,counter strike
7207,Факультет высшего профессионального образования,ИТА ЮФУ (бывш. ТТИ ЮФУ),программирование библия футбол волейбол теннис...
5375,Физический,БГУ,авто вести программирование
9121,Механико-математический,ПГНИУ (ПГУ),программирование лыжа поход
4842,Математико-механический (ИМКН),УрФУ им. первого Президента России Б. Н. Ельцина,спортивный программирование гитара футбол


#BERT SentenceTransformer model (ru version)

In [15]:
! pip install sentence_transformers

In [16]:
import torch

from sentence_transformers import SentenceTransformer, util

In [111]:
# Load pre-trained BERT model
BERT_model = SentenceTransformer('DeepPavlov/rubert-base-cased')

# Compute embeddings for the text column
# text_embeddings = BERT_model.encode(df_copy['combined'].tolist(), convert_to_tensor=True)

# Load embeddings for the text column
text_embeddings = torch.load(home_dir+'text_embeddings_tensor.pt')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [115]:
def get_BERT_recommendations():
    recommendations = df_copy.loc[:, ['faculty_name', 'actual_university_name', 'combined']]

    # Compute embeddings for the introduced text
    query_embedding = BERT_model.encode(user_input, convert_to_tensor=True).type(torch.float64)

    # Compute similarity scores
    cosine_scores = util.pytorch_cos_sim(query_embedding, text_embeddings[df_copy.index.values])

    recommendations = recommendations.loc[cosine_scores.sort(descending=True).indices[0, :top_n].tolist(), :]

    return recommendations

In [117]:
get_BERT_recommendations()

,faculty_name,actual_university_name,combined
2810,Факультет математики и информатики,ЛГУ им. А. С. Пушкина,жить создавать различный свой собственный прое...
6686,Высшая школа управления и инноваций,МГУ,скриптереть написать лаг быть создание сервер ...
42003,Инженерный,БарГУ,кринженер правило волшебник закон девятка маши...
11443,Механико-математический,ННГУ им. Н. И. Лобачевского,наговорить наврать врать быть сам приходить уз...
14328,Факультет международных экономических отношений,МГИМО МИД России,посещать школузаниматься квн команда smileь за...
59318,Реклама и связи с общественностью,ММУ (МУМ),уйти универ курс всё делать делать кайф работа...
48831,Металлургический,ЧГУ,личный спрашивать личка stalker лазейка игра с...
35785,Юридический,ВГУ,русыйглаз зелёныелюбить играть комп школьник з...
36795,Факультет энергетики и информационных систем (...,РИИ,коммуникабильный легко адаптироваться любой ср...
39249,Электротехнический,КНУ (КТУ),клиент сеть microsoft служба доступ файл настр...


#WORD2VEC recommendations

In [18]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# Загружаем предобученную модель Word2Vec
word2vec_model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [97]:
def text_to_vector(text):
    vector = np.zeros(word2vec_model.vector_size)
    words = word_tokenize(text)
    for word in words:
        if word in word2vec_model.key_to_index:
            vector += word2vec_model[word]
    return vector

# Преобразуем текстовые данные в векторы Word2Vec
def get_word2vec_recommendations():
    recommendations = df_copy.loc[df_copy['combined']!='', ['faculty_name', 'actual_university_name', 'combined']].reset_index(drop=True)

    text_vectors = recommendations['combined'].apply(lambda x: text_to_vector(x))
    text_vectors = np.stack(text_vectors.values)
    user_vectors = text_to_vector(user_input)
    user_vectors = user_vectors.reshape(1, -1)

    # вычисление косинусного сходства между пользователями на основе векторов Word2Vec признаков
    cosine_sim_user_word2vec = linear_kernel(user_vectors, text_vectors)

    # получение индекса пользователя с наибольшим косинусным сходством
    similar_user_indices = cosine_sim_user_word2vec.argsort(axis=1)[0, :-top_n-1:-1]

    # получение рекомендаций университета и факультета на основе наиболее похожего пользователя
    recommendations = recommendations.loc[similar_user_indices, :]

    return recommendations

In [98]:
get_word2vec_recommendations()

,faculty_name,actual_university_name,combined
21388,Экономики и управления бизнесом,ДУЭП им. А. Нобеля,wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww qweqwe...
7122,Биоинженерии и биоинформатики,МГУ,год рост брюнетка симпатичный художественный л...
7124,Экономика,РГГУ,год восток запад западный ветер ясный погода п...
7125,Радиоэлектронники и информатики,РГАТУ (бывш. РАТИ) им. Соловьева,женщина харизма пауло коэльо прятки фотография...
7126,Высшая школа бизнеса,МГУ,год любить дискатекилюбить гулять друзьямиигра...
7127,Высшая школа физиков МИФИ-ФИАН,НИЯУ МИФИ,активнаязадорнаялюбить семья депутат молодёжны...
7128,Юридический,НГУЭУ,исаак левитан искусство видеть мир partical ci...
7129,Автомобильного транспорта,СГУВТ (НГАВТ),счастливый мама жена декретный отпуск замечате...
7130,Автомобильный,МГИУ,психически неуровновешать человеккоторый перет...
7131,Энергетики и электрификации,КубГАУ им. И. Т. Трубилина,главный тарелочка последний кольценосец еськов...


#DOC2VEC recommendations

In [21]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

In [99]:
def get_doc2vec_recommendations():
    recommendations = df_copy.loc[df_copy['combined']!='', ['faculty_name', 'actual_university_name', 'combined']].reset_index(drop=True)

    corpus = recommendations['combined'].apply(word_tokenize)
    corpus = [TaggedDocument(words, [idx]) for idx, words in enumerate(corpus)]

    doc2vec_model = Doc2Vec(corpus, min_count=0)

    results = doc2vec_model.dv.most_similar(positive=[doc2vec_model.infer_vector([user_input])], topn=top_n+1)
    idxs = [idx for idx, score in results]

    recommendations = recommendations.loc[idxs, :]

    return recommendations

In [100]:
get_doc2vec_recommendations()

,faculty_name,actual_university_name,combined
19145,1-й факультет (Конструкций ракет-носителей и к...,ВКА им. Можайского,сумерки гулять ночной город
5892,Заочного обучения,МарГУ,любить девчёнка тачка скорость секс девушка ав...
6417,Вокально-режиссерский,СПбГК им. Римского-Корсакова,rap самый ужасный жизнь читать rap жизнь смерт...
13683,Биолого-почвенный,ВГУ,вредный противный музыка кино прогулка друг
20443,АСУ,МГУТУ им. К. Г. Разумовского (ПКУ) (бывш. МГТА...,отдых солнце песок
16981,Международных отношений,МГИМО МИД России,любить девочка автомобиль отдых рубеж
2709,Автомобили и автомобильное хозяйство,АлтГТУ им. Ползунова,простой репер нечитать доганя керпич rap rap r...
18478,Гуманитарный факультет,СПбГУ,гей стесняться парень добавляться друг мальчик
678,Финансово-кредитный,КФУ (бывш. КГУ им. Ульянова-Ленина),диджей искать девушка красивую больший ресните...
12593,Заочный факультет,НГУЭУ,белый ночь out the park baseball очень любить ...
